In [1]:
#调库 析晶程序判别
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import copy

In [2]:
frame_num = 150
file_path = r'C:\Users\zhang\Desktop\13.0厚度xyz文件\test\30out10ns_2000p_out.xyz'
creat_new_frame = 1

In [3]:
#定义距离函数
def cal_distance(x1,y1,z1,x2,y2,z2):
    dis = ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**(1/2)
    return dis

In [4]:
########参数列表#############
max_neigh = 6
max_dis = 6.0


In [5]:
#读取文本函数，返回直接d选帧
def get_data(path,frame_num):
    with open(path) as f:
        str_data = f.read()
    data = str_data.split()
    ###选帧#########
    #此处写个程序选帧
    atoms_num = int(data[0])
    frame_len = atoms_num*4+2
    data_len =len(data)
    max_frame = data_len // frame_len
    
    temp = []
    temp = data[frame_len*frame_num:frame_len*(frame_num+1)]

    #暂时先自行处理了表头
    temp = temp[2:]
    frame_list = []
    for i in range(atoms_num):
        frame_list.append([temp[i*4+0],float(temp[i*4+1]),float(temp[i*4+2]),float(temp[i*4+3])])
    
    return frame_list

In [6]:
class point:
    def __init__(self,name,x,y,z):
        #自身坐标
        self.x=x
        self.y=y
        self.z=z
        self.name=name
        self.cry=0
        #记录最近的几个点 每个点按顺序记录下xyz和dis 
        self.neighbor_atom=[]
        #记录与最近的几个点的角度 
        self.angle=[]
    
    #定义类中的替换函数  如果这个距离比原有的小，就替换掉原有点的记录
    def show(self):
        print("name: ",self.name," xyz: ",self.x," ",self.y," " ,self.z)
        print("neighbor: ")
        for i in self.neighbor_atom:
            print(i)
        print("angle: ")
        for i in self.angle:
            print(i)
            
    def sub(self,id,x,y,z,dis,max_dis):
        
        #为空的话直接放进去
        if(len(self.neighbor_atom)==0):
            self.neighbor_atom.append([id,x,y,z,dis])
            return
        #大多数点是比表中最大的点距离大的 所以大的直接返回
        if(dis>max_dis):
            return
        #如果需要调整,那么先插入表尾，再排序，再删除最后一个节点就行
        else:
            self.neighbor_atom.append([id,x,y,z,dis])
            self.neighbor_atom.sort(key=lambda xx:xx[-1])
            #如果表满了则删除最后一个 或者最后一个大于了max dis
            if(len(self.neighbor_atom)>max_neigh or self.neighbor_atom[-1][-1]>max_dis):
                del self.neighbor_atom[-1]
    
    def filter_neighbor(self,max_dis):
        self.neighbor_atom.sort(key=lambda xx:xx[-1])
        # 仅当nei的数量比较大时才筛选
        if(len(self.neighbor_atom)<=1):
            return
        # 这才筛选 以免很多原子没有nei
        for i in range(len(self.neighbor_atom)-1,-1,-1):
            if(self.neighbor_atom[-1][-1]>max_dis):
                self.neighbor_atom.pop()
    
    #计算以中心点为顶点，距离最近的任意两点为边的角的角度值
#     def cal_angle(self):
#         for i in range(len(self.neighbor_atom)):
#             for j in range(i+1,len(self.neighbor_atom)):
#                 #此时便有了point nei[i] nei[j]三个点
#                 #定义两个空间向量
#                 x1,y1,z1 = (self.neighbor_atom[i][0]-self.x),(self.neighbor_atom[i][1]-self.y),(self.neighbor_atom[i][2]-self.z)
#                 x2,y2,z2 = (self.neighbor_atom[j][0]-self.x),(self.neighbor_atom[j][1]-self.y),(self.neighbor_atom[j][2]-self.z)
#                 #两个向量的夹角
#                 cos_b = (x1*x2 + y1*y2 + z1*z2) / (math.sqrt(x1**2 + y1**2 + z1**2) *(math.sqrt(x2**2 + y2**2 + z2**2)))
#                 B = math.degrees(math.cos(cos_b)) 
#                 self.angle.append(B)
    def cal_angle(self):
        self.angle.clear()
        for i in range(len(self.neighbor_atom)):
            for j in range(i+1,len(self.neighbor_atom)):
                a = [self.neighbor_atom[i][1], self.neighbor_atom[i][2], self.neighbor_atom[i][3]]
                
                b = [self.x, self.y, self.z]
                
                c=[self.neighbor_atom[j][1], self.neighbor_atom[j][2], self.neighbor_atom[j][3]]
                
                temp = fun_angle(a,b,c)
                self.angle.append(temp)
        self.angle.sort()
    #最后还应该有一个判断是不是晶体的程序
    def judge_cry(self):
        pass

In [7]:
def classify_atom(data,atom_type):
    ans_list = []
    a_type =len(atom_type)
    for i in range(a_type):
        ans_list.append([])

    for i in data:
        for j in range(a_type):
               if(i[0]==atom_type[j]):
                    ans_list[j].append(point(i[0],i[1],i[2],i[3]))
    return ans_list 

# 主函数

In [8]:
############主函数##############


new_name = r'C:\Users\zhang\Desktop\frame_ana_cry.xyz'
if(creat_new_frame==1):
    new_name = "C:\\Users\zhang\\Desktop\\"
    new_name += str(frame_num) + 'frame.xyz'
    
data = get_data(file_path,frame_num)

####获取元素分类列表
(nb_list,li_list,o_list,si_list) = classify_atom(data,['Nb','Li','O','Si'])


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zhang\\Desktop\\13.0厚度xyz文件\\test\\30out10ns_2000p_out.xyz'

In [ ]:
##此处分为两种情况，一个是计算Nb-Nb之间的距离，一个是计算Nb-O之间的距离
#这里计算Nb-Nb距离
for i in range(len(nb_list)):
    for j in range(i+1,len(nb_list)):
        dis = cal_distance(nb_list[i].x, nb_list[i].y, nb_list[i].z,nb_list[j].x,nb_list[j].y,nb_list[j].z)
        #通过计算各个点的距离  如果距离小了就放入列表中
        nb_list[i].sub(j,nb_list[j].x, nb_list[j].y, nb_list[j].z, dis, max_dis)
        nb_list[j].sub(i,nb_list[i].x, nb_list[i].y, nb_list[i].z, dis, max_dis)

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
@Time     : 2021/8/10 9:16
@Author   : Allen Pen(Haijun Peng)
@E-mail   : shengyutou@outlook.com
NOTE:
"""
import math


def fun_angle(point_a, point_b, point_c):
    """
    根据三点坐标计算夹角

                  点a
           点b ∠
                   点c

    :param point_a、point_b、point_c: 数据类型为list,二维坐标形式[x、y]或三维坐标形式[x、y、z]
    :return: 返回角点b的夹角值


    数学原理：
    设m,n是两个不为0的向量，它们的夹角为<m,n> (或用α ,β, θ ,..,字母表示)

    1、由向量公式：cos<m,n>=m.n/|m||n|

    2、若向量用坐标表示，m=(x1,y1,z1), n=(x2,y2,z2),

    则,m.n=(x1x2+y1y2+z1z2).

    |m|=√(x1^2+y1^2+z1^2), |n|=√(x2^2+y2^2+z2^2).

    将这些代入②得到：

    cos<m,n>=(x1x2+y1y2+z1z2)/[√(x1^2+y1^2+z1^2)*√(x2^2+y2^2+z2^2)]

    上述公式是以空间三维坐标给出的，令坐标中的z=0,则得平面向量的计算公式。

    两个向量夹角的取值范围是：[0,π].

    夹角为锐角时，cosθ>0；夹角为钝角时,cosθ<0.

    """
    a_x, b_x, c_x = point_a[0], point_b[0], point_c[0]  # 点a、b、c的x坐标
    a_y, b_y, c_y = point_a[1], point_b[1], point_c[1]  # 点a、b、c的y坐标

    if len(point_a) == len(point_b) == len(point_c) == 3:
        # print("坐标点为3维坐标形式")
        a_z, b_z, c_z = point_a[2], point_b[2], point_c[2]  # 点a、b、c的z坐标
    else:
        a_z, b_z, c_z = 0,0,0  # 坐标点为2维坐标形式，z 坐标默认值设为0
        # print("坐标点为2维坐标形式，z 坐标默认值设为0")

    # 向量 m=(x1,y1,z1), n=(x2,y2,z2)
    x1,y1,z1 = (a_x-b_x),(a_y-b_y),(a_z-b_z)
    x2,y2,z2 = (c_x-b_x),(c_y-b_y),(c_z-b_z)

    # 两个向量的夹角，即角点b的夹角余弦值
    cos_b = (x1*x2 + y1*y2 + z1*z2) / (math.sqrt(x1**2 + y1**2 + z1**2) *(math.sqrt(x2**2 + y2**2 + z2**2))) # 角点b的夹角余弦值
    B = math.degrees(math.acos(cos_b)) # 角点b的夹角值
    return B

In [ ]:
for i in nb_list:
    i.cal_angle()

In [ ]:
#################现在把计算后得来的数据都复制好，之后再对数据处理就不要对原始数据进行更改了

copy_list_nbnb =nb_list.copy() 

In [ ]:
x,y = [],[]
for i in copy_list_nbnb:
    i.cal_angle()
    for j in i.angle:
        x.append(i.x)
        y.append(j)

plt.scatter(x,y,s=5)

In [ ]:
for i in copy_list_nbnb:
    i.filter_neighbor(5.0)
x,y = [],[]
for i in copy_list_nbnb:
    i.cal_angle()
    for j in i.angle:
        x.append(i.x)
        y.append(j)

plt.scatter(x,y,s=5)

In [ ]:
#分析程序
#大概意思就是 一个点80%的angle只有在70-110和165-180附近才算是cry
for i in copy_list_nbnb:
    count = 0
    sum_all = 0
    i.cry = 0
    cry_ratio=0
    for j in i.angle:
        if(70<j<110 or 165<j<180):
            count+=1
        sum_all+=1
    if(sum_all!=0):   
        cry_ratio = count/sum_all
    if(cry_ratio>0.8):
        i.cry=1

#注意有的sum为0  一会儿去观察一下为什么

x,y=[],[]
cry_x,cry_y = [],[]
for i in copy_list_nbnb:
    if(i.cry==1):
        cry_x.append(i.x)
        cry_y.append(i.z)
    else:
        x.append(i.x)
        y.append(i.z)

        
# 画图 
plt.scatter(x,y,s=5,c='b')
plt.scatter(cry_x,cry_y,s=5,c='r')

In [ ]:
#分析程序
#大概意思就是 一个点80%的angle只有在70-110和165-180附近才算是cry
for i in copy_list_nbnb:
    i.cry=0
for i in copy_list_nbnb:
    count = 0
    sum_all = 0
    i.cry = 0
    cry_ratio = 0
    for j in i.angle:
        if(70<j<110 or 165<j<180):
            count+=1
        sum_all+=1
    if(sum_all!=0):   
        cry_ratio = count/sum_all
    if(cry_ratio>0.8):
        i.cry=1
        
        
for i in nb_list:
    if(i.cry==0):
        flag = 0
        sum_num = 0
        # 如果四周全为cry 则其为cry
        for j in i.neighbor_atom:
            if(copy_list_nbnb[j[0]].cry==1):
                flag+=1
            sum_num+=1
        # 如果四周的cry超六成 则其视为cry
        if(sum_num!=0 and flag/sum_num > 0.6):
            i.cry=1
    # 反之cry=1
    if(i.cry==1):
        flag = 0
        sum_num = 0
        # 如果四周全为no cry 则其为no cry
        for j in i.neighbor_atom:
            if (copy_list_nbnb[j[0]].cry == 0):
                flag += 1
            sum_num += 1
        # 如果四周的no cry超六成 则其视为cry
        if (sum_num!=0 and flag / sum_num > 0.7):
            i.cry = 0

x,y=[],[]
cry_x,cry_y = [],[]
for i in copy_list_nbnb:
    if(i.cry==1):
        cry_x.append(i.x)
        cry_y.append(i.z)
    else:
        x.append(i.x)
        y.append(i.z)

        
# 画图 
plt.scatter(x,y,s=5,c='b')
plt.scatter(cry_x,cry_y,s=5,c='r')

In [ ]:
#看文件是否存在 存在也删了 都要重建
if not os.path.exists(new_name):
    os.system(r'touch {}'.format(new_name))
else:
    os.remove(new_name)
    os.system(r'touch {}'.format(new_name))

In [ ]:
#先把文件头写上去
with open(new_name,'a') as file:
        file.write("{} \n".format(6200))
        file.write("{} \n".format("Atoms"))

#如果cry标志位为1  那么写入的原子name为Ag
with open(new_name,'a') as file:
    for i in copy_list_nbnb:
        if(i.cry==1):
            file.write("{}    {}    {}    {}\n".format("Ag",i.x,i.y,i.z))
        else:
            file.write("{}    {}    {}    {}\n".format(i.name,i.x,i.y,i.z))
#再写入其他原子
with open(new_name,'a') as file:
    for i in o_list:
        file.write("{}    {}    {}    {}\n".format(i.name,i.x,i.y,i.z))
    for i in si_list:
        file.write("{}    {}    {}    {}\n".format(i.name,i.x,i.y,i.z))
    for i in li_list:
        file.write("{}    {}    {}    {}\n".format(i.name,i.x,i.y,i.z))

In [ ]:
count = 0
for i in copy_list_nbnb:
    if(i.cry==1):
        count+=1
print("crystalization atom number:",count)

In [ ]:
count=0
for i in copy_list_nbnb:
    count+=1
print(count)